In [1]:
# add local directory to import path
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
#### --- Standard imports ------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import HTML

# local import
import nflplot

In [3]:
from importlib import reload
reload(nflplot)

<module 'nflplot' from 'c:\\Users\\pschl\\Documents\\data_science\\kaggle_nfl_bdb_2024\\nflplot.py'>

**This notebook assumes that the source data is in a "csv" subfolder relative to this notebook**

In [4]:
week = 1
track_df = pd.read_csv(f'csv/tracking_week_{week}.csv')
play_df = pd.read_csv('csv/plays.csv')
game_df = pd.read_csv('csv/games.csv')
player_df= pd.read_csv('csv/players.csv')
tackle_df = pd.read_csv('csv/tackles.csv')

In [8]:
solo_tackle_plays = tackle_df.query('tackle==1').loc[:, ['gameId','playId']].drop_duplicates()
len(solo_tackle_plays)

In [35]:
print('Number of plays in week 1 with a solo tackle:')
len(play_df.merge(game_df.loc[game_df.week==1, 'gameId'], on='gameId', how='inner')
    .merge(solo_tackle_plays, on=['gameId','playId'], how='inner')
    )

Number of plays in week 1 with a solo tackle:


1213

In [30]:
pm10_df = (track_df.groupby(['gameId','playId','frameId']).head(1)
           .sort_values(['gameId','playId','frameId'], ascending=[True, True, True])
           .groupby(['gameId','playId'])
           .tail(10)
           .loc[:, ['gameId','playId','frameId']]
           )
pm10_df

,gameId,playId,frameId
12,2022090800,56,13
13,2022090800,56,14
14,2022090800,56,15
15,2022090800,56,16
16,2022090800,56,17
...,...,...,...
1406268,2022091200,3826,49
1406269,2022091200,3826,50
1406270,2022091200,3826,51
1406271,2022091200,3826,52


In [36]:
print('Events during solo tackle plays, in the last 10 frames of the play:')
(
    track_df.groupby(['gameId','playId','frameId']).head(1)
    .merge(pm10_df, on=['gameId','playId','frameId'], how='inner')
    .merge(solo_tackle_plays, on=['gameId','playId'], how='inner')
    .loc[:, 'event']
    .value_counts())

Events during solo tackle plays, in the last 10 frames of the play:


event
tackle                      974
out_of_bounds               203
first_contact               199
pass_outcome_caught          73
fumble                       25
pass_arrived                 20
qb_slide                     14
fumble_defense_recovered      1
Name: count, dtype: int64